In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_table("http://www.econometrics.com/comdata/gujarati/data_15.7.shd", header = None,
                    delim_whitespace = True,
                    names = ["Fumador","Edad","Escolaridad","Ingresos","Pcigs79"]) # Cargar de internet

In [ ]:
Y = data["________"]
X = data[["_____","__________","_____","________-"]]

In [ ]:
MLPMargEff = _________.params 

In [ ]:
PrediccionMLP = _____________.predict()
PrediccionMLP = pd.DataFrame(__________, columns = ["Prediccion MLP"])

In [ ]:
ModeloLogit = sm.______(_______, sm.add_constant(____-))
ResultadosLogit = ____________
print(ResultadosLogit.summary())

In [ ]:
LogitMargEff = ResultadosLogit.____________
print(LogitMargEff.summary())

In [ ]:
LogitMargEff = ResultadosLogit.___________(at = "______")
print(LogitMargEff.summary())

**Ejercicio modelo LOGIT con escolaridad únicamente de más de 10 años**

**Modelo Probit**

In [ ]:
ModeloProbit = sm._______(____,sm.add_constant(___))
ResultadosProbit = ____________.fit()
print(ResultadosProbit.summary())

In [ ]:
ProbitMarEff = ________.___________()
print(ProbitMarEff.summary())

In [ ]:
ProbitMarEff = ResultadosProbit.______(at = "_____")
print(ProbitMarEff.summary())

In [ ]:
from statsmodels.iolib.summary2 import summary_col

In [ ]:
Tabla = summary_col(results=[_______,________, _________],
                   model_names = [_______,_______,_______], stars= True)
print(Tabla)

In [ ]:
Coeficientes = ["Edad","Escolaridad","Ingresos","Pcigs79"]
EfectosLogit = pd.DataFrame()
EfectosProbit = pd.DataFrame()
EfectosMLP = pd.DataFrame()

In [ ]:
pd.concat([___________________], axis = 1)